# Load Dataframe via Trino

In [22]:
from kfp.components import create_component_from_func, OutputPath

%load_ext lab_black

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def load_dataframe_via_trino(
    query: str,
    dataframe_file: OutputPath(str),
    columns_query: str = None,
    host: str = "trino.trino",
    port: int = 8080,
    user: str = "anybody",
    catalog: str = None,
    schema: str = None,
):
    """
    Load a Pandas Dataframe using Trino as SQL client.

            Parameters:
                    query: An ANSI SQL compliant query for data, as supported by Trino. Queries can either use explicit or implicit references to schemata and catalogs. In the implicit case, the parameters catalog and schema must be set. Example: "SELECT * FROM transactions OFFSET 20".
                    columns_query: An ANSI SQL compliant "SHOW COLUMNS" query for data columns, as supported by Trino. Queries can either use explicit or implicit references to schemata and catalogs. In the implicit case, the parameters catalog and schema must be set. If not set, generic column names are used. Example: "SHOW COLUMNS FROM postgresql.public.transactions".
                    host: Host of the trino installation, typically the trino service in the trino namespace. Example:  "trino.trino".
                    port: Trino service port. Example: "8080".
                    user: Sets the query context to the given user. The user needs permissions to access the targeted catalog and schema. Example: "anybody".
                    catalog: Sets the query context to the given catalog. If None, the query must explicitly reference to schemata and catalogs. If set, also a schema must be set. Example: "postgresql".
                    schema: Sets the query context to the given schema. If None, the query must explicitly reference to schemata and catalogs. If set, also a catalog must be set. Example: "public".
            Returns:
                    dataframe_file: A Pandas dataframe containing the query results.
    """
    import logging
    import pandas as pd
    import sys
    from trino.dbapi import Connection

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )

    if (catalog is not None and schema is None) or (
        catalog is None and schema is not None
    ):
        raise Exception(
            f"If you set one, you need to set both: catalog={catalog} but schema={schema}!"
        )

    logging.info("Establishing Trino connection...")
    with Connection(
        host=host,
        port=port,
        user=user,
        catalog=catalog,
        schema=schema,
    ) as conn:
        cursor = conn.cursor()

        logging.info("Querying data...")
        cursor.execute(query)
        dataframe = pd.DataFrame(cursor.fetchall())
        logging.info(f"Retrieved {len(dataframe)} rows.")

        if columns_query is not None:
            logging.info("Querying column names...")
            cursor.execute(columns_query)
            columns_dataframe = pd.DataFrame(cursor.fetchall())
            columns = columns_dataframe[0].values.tolist()
            dataframe.columns = columns
            logging.info(f"Found columns: {columns}")

    # Feather outperforms Pickle & Parquet
    # See https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d
    dataframe.to_feather(dataframe_file)
    logging.info("Finished.")


load_dataframe_via_trino_comp = create_component_from_func(
    func=load_dataframe_via_trino,
    output_component_file="component.yaml",
    base_image=BASE_IMAGE,
)

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
